In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gc
gc.enable()
gc.collect()

pd.options.display.max_columns = 100

%matplotlib inline

In [4]:
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')
train.shape, test.shape

FileNotFoundError: [Errno 2] File ./training.csv does not exist: './training.csv'

In [ ]:
#for col in ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 
#            'ChannelId']:
#    train[col] = train[col].str.extract('(\d+)')
#    train[col] = train[col].astype('int')
    
#    test[col] = test[col].str.extract('(\d+)')
#    test[col] = test[col].astype('int')

In [ ]:
train['TransactionStartTime'] = train['TransactionStartTime'].str.replace('T', ' ')
train['TransactionStartTime'] = train['TransactionStartTime'].str.replace('Z', '')

test['TransactionStartTime'] = test['TransactionStartTime'].str.replace('T', ' ')
test['TransactionStartTime'] = test['TransactionStartTime'].str.replace('Z', '')

In [ ]:
#train['Date'] = train['TransactionStartTime'].str.extract('(\d+-\d+-\d+)T')
#train['Time'] = train['TransactionStartTime'].str.extract('(\d+:\d+:\d+)Z')

In [ ]:
train['TransactionStartTime'] = pd.to_datetime(train['TransactionStartTime'], infer_datetime_format=True) 
test['TransactionStartTime'] = pd.to_datetime(test['TransactionStartTime'], infer_datetime_format=True) 

In [ ]:
train['Year'] = train['TransactionStartTime'].dt.year
train['Month'] = train['TransactionStartTime'].dt.month
train['Day'] = train['TransactionStartTime'].dt.day
train['Hour'] = train['TransactionStartTime'].dt.hour
train['Minute'] = train['TransactionStartTime'].dt.minute
train['Seconds'] = train['TransactionStartTime'].dt.second
train['week'] = train['TransactionStartTime'].dt.week
train['weekday'] = train['TransactionStartTime'].dt.weekday

test['Year'] = test['TransactionStartTime'].dt.year
test['Month'] = test['TransactionStartTime'].dt.month
test['Day'] = test['TransactionStartTime'].dt.day
test['Hour'] = test['TransactionStartTime'].dt.hour
test['Minute'] = test['TransactionStartTime'].dt.minute
test['Seconds'] = test['TransactionStartTime'].dt.second
test['week'] = test['TransactionStartTime'].dt.week
test['weekday'] = test['TransactionStartTime'].dt.weekday

In [ ]:
train.loc[train['Amount'] > 0, 'Debit'] = 1
train.loc[train['Amount'] < 0, 'Debit'] = 0

test.loc[test['Amount'] > 0, 'Debit'] = 1
test.loc[test['Amount'] < 0, 'Debit'] = 0

train['Debit'] = train['Debit'].astype('int')
test['Debit'] = test['Debit'].astype('int')

In [ ]:
all_df = pd.concat([train, test]).reset_index(drop=True)
all_df = all_df.sort_values(by='TransactionStartTime').reset_index(drop=True)
all_df.shape

In [ ]:
additional_cols = ['CustomerId', 'AccountId', 'TransactionStartTime', 'CountryCode']
new_df = all_df[['TransactionId']+ additional_cols]

In [ ]:
for col in ['Value', 'Day', 'Seconds', 'Amount']:
    gdf = all_df.groupby('CustomerId')[col].mean().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_mean']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')
    
for col in ['Value', 'Day', 'Seconds', 'Amount', 'Debit']:
    gdf = all_df.groupby('CustomerId')[col].sum().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_sum']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')
    
#for col in ['Value', 'Seconds', 'week', 'Amount']:
#    gdf = all_df.groupby('CustomerId')[col].cumsum().reset_index()
#    gdf.columns = ['CustomerId', 'Customer_'+col+'_cumsum']
#    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')

for col in ['AccountId', 'BatchId', 'SubscriptionId', 'ChannelId', 'ProductId', 'ProviderId']:
    gdf = all_df.groupby('CustomerId')[col].nunique().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_nunique']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')

In [ ]:
new_df['Customer_cumcount_seconds'] = all_df.groupby('CustomerId')['Seconds'].cumcount().values
new_df['Customer_cumsum_day'] = all_df.groupby('CustomerId')['Day'].cumsum().values
new_df['Customer_cumsum_month'] = all_df.groupby('CustomerId')['Month'].cumsum().values

#all_df['prev_trans_date'] = all_df.groupby('CustomerId')['Seconds'].shift(1)
#new_df['time_gap_prev'] = all_df['Seconds'] - all_df['prev_trans_date']

#all_df['next_trans_date'] = all_df.groupby('CustomerId')['Seconds'].shift(-1)
#new_df['time_gap_next'] = all_df['Seconds'] - all_df['next_trans_date']

#all_df['prev_trans_date2'] = all_df.groupby('CustomerId')['Seconds'].shift(2)
#new_df['time_gap_prev2'] = all_df['Seconds'] - all_df['prev_trans_date2']

#all_df['next_trans_date2'] = all_df.groupby('CustomerId')['Seconds'].shift(-2)
#new_df['time_gap_next2'] = all_df['Seconds'] - all_df['next_trans_date2']

In [ ]:
gdf = pd.pivot_table(all_df, index="CustomerId", columns="Year", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="Month", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="week", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="weekday", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

In [ ]:
gdf = pd.pivot_table(all_df, index="CustomerId", columns="ChannelId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="ProviderId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="ProductId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

In [ ]:
new_df = new_df.drop(additional_cols, axis=1)
new_df.head()

In [ ]:
new_df.to_csv('Feat_v4.csv', index=False)